In [1]:
!pip install elasticsearch
!pip install sentence-transformers

In [2]:
import json
from elasticsearch import Elasticsearch

es = Elasticsearch( "http://elasticsearch:9200")


In [3]:
index_name = 'drivers_descriptions'

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-large')

/opt/conda/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Query

In [4]:
res = es.search(index=index_name, body={
    'query': {
        'bool': {
            'must': [
                {'match': {'wikipedia_description': 'youngest world champion'}},
                {'match': {'wikipedia_description': 'youngest race winner'}}
            ]
        }
    }
})

res = es.search(index=index_name, body={
    'query': {
        'match': {
            'wikipedia_description': 'youngest race winner'
        }
    }
})

In [10]:
res = es.search(index=index_name, body={
    'query': {
        'bool': {
            'must': [
                {'match': {'wikipedia_description': 'le mans'}},
                # {'match': {'wikipedia_description': 'UNICEF'}},
                {'match': {'wikipedia_description': 'youngest world champion'}},
                {'match': {'wikipedia_description': 'youngest race winner'}}
            ]
        }
    }
})
print(json.dumps(
    [ {"driverId": x["_source"]["driverId"], "wikipedia_description": x["_source"]["wikipedia_description"]} 
      for x in res.body["hits"]["hits"] ],
    indent=4
))

[
    {
        "driverId": 101,
        "wikipedia_description": "David Philip Brabham (born 5 September 1965) is an Australian racing driver and one of the most successful and experienced specialists in sports car racing. He has won three international Sports Car series and is one of four Australians to have won the Le Mans 24 Hour sports car race, winning the event in 2009. Brabham won the American Le Mans Series in 2009 and 2010. He also competed in Formula One, racing for the Brabham and Simtek teams in 1990 and 1994, respectively. Brabham is the youngest son of three-time Formula One world champion Sir Jack Brabham, brother to Geoff Brabham and Gary Brabham. He is also brother-in-law to Mike Thackwell, father to Sam Brabham and uncle to Matthew Brabham.\n"
    },
    {
        "driverId": 4,
        "wikipedia_description": "Fernando Alonso D\u00edaz (Spanish pronunciation: [fe\u027e\u02c8nando a\u02c8lonso \u02c8\u00f0i.a\u03b8] \u24d8; born 29 July 1981) is a Spanish racing dri

In [39]:
query_sentence = "query: pilotos que corrieron en le mans y en indianapolis"
query_vector = model.encode([query_sentence])[0]

parameters = {
    "field":"wikipedia_description_vector",
    "query_vector": query_vector,
    "k": 100,
    "num_candidates":100
}

res = es.search(index=index_name, knn=parameters, size=100)

print(json.dumps(
    [ {"driverId": x["_source"]["driverId"], "wikipedia_description": x["_source"]["wikipedia_description"] }
      for x in res.body["hits"]["hits"] ],
    indent=4
))    

[
    {
        "driverId": 296,
        "wikipedia_description": "Fran\u00e7ois Migault (4 December 1944\u00a0\u2013 29 January 2012) was a racing driver from Le Mans, France.[2] He participated in 16 Formula One World Championship Grands Prix, debuting on 13 August 1972, but scored no championship points.\n"
    },
    {
        "driverId": 7,
        "wikipedia_description": "S\u00e9bastien Olivier Bourdais[1] (French pronunciation: [sebastj\u025b\u0303 \u0254livje bu\u0281d\u025b]; born 28 February 1979) is a French professional racing driver who currently races in the IMSA WeatherTech SportsCar Championship for Cadillac Chip Ganassi Racing in the GTP category. He is one of the most successful drivers in the history of American open-wheel car racing, having won 37 races.  He won four successive championships in the Open-Wheel Racing Series-sanctioned Champ Car World Series from 2004 to 2007. Later he competed at the IndyCar Series from 2011 to 2021. He also entered 27  races in For